In [1]:
import pandas as pd
import glob
import os

data_dir = '../ml/data/'   

print("Folder:", os.path.abspath(data_dir))

listings_files = sorted(glob.glob(data_dir + 'clean_sales_listings_*.csv'))
history_files  = sorted(glob.glob(data_dir + 'clean_sales_history_*.csv'))
agents_files   = sorted(glob.glob(data_dir + 'clean_sales_agents_*.csv'))
offices_files  = sorted(glob.glob(data_dir + 'clean_sales_offices_*.csv'))

print("\nNewest listings file:", listings_files[-1] if listings_files else "not found")
print("Newest history file: ", history_files[-1]  if history_files  else "not found")
print("Newest agents file:   ", agents_files[-1]   if agents_files   else "not found")
print("Newest offices file:  ", offices_files[-1]  if offices_files  else "not found")

Folder: c:\Users\A\Desktop\Projects\Alloy Tower\ml\data

Newest listings file: ../ml/data\clean_sales_listings_20260201_134434.csv
Newest history file:  ../ml/data\clean_sales_history_20260201_134435.csv
Newest agents file:    ../ml/data\clean_sales_agents_20260201_134436.csv
Newest offices file:   ../ml/data\clean_sales_offices_20260201_134437.csv


In [2]:
# Use the newest listings file from Chunk 1
listings_path = listings_files[-1]   # this was already found in Chunk 1

print("Loading:", os.path.basename(listings_path))

listings = pd.read_csv(listings_path)

print("\nShape:", listings.shape)
print("Memory usage: ~{:.1f} MB".format(listings.memory_usage(deep=True).sum() / 1024**2))

print("\nColumns (28 total):", listings.columns.tolist())

print("\nFirst 2 rows:\n", listings.head(2).to_string(index=False))

print("\nListed date range:", 
      listings['LISTED_DATE'].min(), "to", listings['LISTED_DATE'].max())

print("\nStatus distribution:\n", listings['STATUS'].value_counts(dropna=False))

Loading: clean_sales_listings_20260201_134434.csv

Shape: (13428, 28)
Memory usage: ~12.4 MB

Columns (28 total): ['LISTING_ID', 'ADDRESS', 'STREET', 'UNIT', 'CITY', 'COUNTY', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'PROPERTY_TYPE', 'BEDROOMS', 'BATHROOMS', 'SQUARE_FOOTAGE', 'LOT_SIZE', 'YEAR_BUILT', 'STATUS', 'CURRENT_PRICE', 'HOA_FEE', 'PRICE_PER_SQ_FT', 'LISTING_TYPE', 'MLS_NUMBER', 'DAYS_ON_MARKET', 'LISTED_DATE', 'REMOVED_DATE', 'CREATED_DATE', 'LAST_SEEN_TS', 'AGENT_ID', 'OFFICE_ID']

First 2 rows:
       LISTING_ID                                     ADDRESS                  STREET UNIT     CITY    COUNTY  ZIP_CODE  LATITUDE  LONGITUDE PROPERTY_TYPE  BEDROOMS  BATHROOMS  SQUARE_FOOTAGE  LOT_SIZE  YEAR_BUILT STATUS  CURRENT_PRICE  HOA_FEE  PRICE_PER_SQ_FT     LISTING_TYPE  MLS_NUMBER  DAYS_ON_MARKET LISTED_DATE  REMOVED_DATE CREATED_DATE            LAST_SEEN_TS         AGENT_ID        OFFICE_ID
043871b3d5cb5961          601 Hillside Dr, Sherman, TX 75090         601 Hillside Dr  NaN

In [3]:

history_path = history_files[-1]
agents_path  = agents_files[-1]
offices_path = offices_files[-1]

print("Loading history :", os.path.basename(history_path))
history = pd.read_csv(history_path)

print("Loading agents  :", os.path.basename(agents_path))
agents = pd.read_csv(agents_path)

print("Loading offices :", os.path.basename(offices_path))
offices = pd.read_csv(offices_path)

print(f"\nLoaded shapes:")
print(f"History:  {history.shape[0]:,} rows × {history.shape[1]} cols")
print(f"Agents:   {agents.shape[0]:,} rows × {agents.shape[1]} cols")
print(f"Offices:  {offices.shape[0]:,} rows × {offices.shape[1]} cols")

# Quick peek at history (most interesting)
print("\nHistory sample columns:", history.columns.tolist())
print("\nFirst 3 history rows:\n", history.head(3).to_string(index=False))

Loading history : clean_sales_history_20260201_134435.csv
Loading agents  : clean_sales_agents_20260201_134436.csv
Loading offices : clean_sales_offices_20260201_134437.csv

Loaded shapes:
History:  8,459 rows × 8 cols
Agents:   7,200 rows × 5 cols
Offices:  3,103 rows × 5 cols

History sample columns: ['LISTING_ID', 'EVENT_DATE', 'EVENT_TYPE', 'EVENT_PRICE', 'LISTING_TYPE', 'LISTED_DATE', 'REMOVED_DATE', 'DAYS_ON_MARKET']

First 3 history rows:
       LISTING_ID EVENT_DATE   EVENT_TYPE  EVENT_PRICE     LISTING_TYPE LISTED_DATE REMOVED_DATE  DAYS_ON_MARKET
726f545750f0bbcb 2025-11-30 sale listing      1299900 New Construction  2025-11-30   2026-01-18              49
726f545750f0bbcb 2025-12-24 sale listing      1179900 New Construction  2025-12-24   2026-01-22              29
13c40b48eb461e70 2025-02-28 sale listing       150000         Standard  2025-02-28   2025-04-16              47


EDA

Join check : Do agent IDs in listings match agents table?
Do listing IDs in history match current listings?

In [5]:
print("=== Quick Join Readiness Check ===\n")

# 1. Agent & Office key overlap
print("Unique AGENT_ID in listings:", listings['AGENT_ID'].nunique())
print("Unique OFFICE_ID in listings:", listings['OFFICE_ID'].nunique())
print("Agents table has:", agents['AGENT_ID'].nunique(), "unique agent_ids")

# 2. History matching to current listings
matching_pct = history['LISTING_ID'].isin(listings['LISTING_ID']).mean() * 100
print(f"\n% of history rows that match a current listing ID: {matching_pct:.1f}%")
print(f"Total history rows: {len(history):,}")
print(f"Matching history rows: {history['LISTING_ID'].isin(listings['LISTING_ID']).sum():,}")

# 3. Example matching IDs (if any)
common = history[history['LISTING_ID'].isin(listings['LISTING_ID'])]['LISTING_ID'].unique()
if len(common) > 0:
    print("\nExample matching LISTING_IDs:", common[:3].tolist())
else:
    print("\nNo overlap in LISTING_ID yet — history may be mostly sold/removed listings.")

=== Quick Join Readiness Check ===

Unique AGENT_ID in listings: 7190
Unique OFFICE_ID in listings: 2994
Agents table has: 7189 unique agent_ids

% of history rows that match a current listing ID: 100.0%
Total history rows: 8,459
Matching history rows: 8,459

Example matching LISTING_IDs: ['726f545750f0bbcb', '13c40b48eb461e70', '279693ea95992d23']


In [6]:
print("=== Basic KPI Summary (Listings Table) ===\n")

# Numeric columns of interest
num_cols = ['CURRENT_PRICE', 'PRICE_PER_SQ_FT', 'HOA_FEE', 'DAYS_ON_MARKET', 'SQUARE_FOOTAGE', 'BEDROOMS', 'BATHROOMS']

print("Overall statistics:")
print(listings[num_cols].describe().round(2))

print("\nTotal listings:", len(listings))
print("Number of unique counties:", listings['COUNTY'].nunique())
print("\nTop 5 counties by listing count:\n", listings['COUNTY'].value_counts().head())

print("\nProperty type breakdown (top 5):\n", listings['PROPERTY_TYPE'].value_counts().head())

print("\nMissing % in key columns:")
print(listings[['PRICE_PER_SQ_FT', 'HOA_FEE', 'DAYS_ON_MARKET', 'CURRENT_PRICE']].isna().mean().round(4) * 100)

=== Basic KPI Summary (Listings Table) ===

Overall statistics:
       CURRENT_PRICE  PRICE_PER_SQ_FT   HOA_FEE  DAYS_ON_MARKET  \
count       13428.00         11146.00  13428.00        13428.00   
mean       614683.85           276.55     78.46          123.24   
std       1047243.72           293.83    196.41          148.85   
min          7500.00             1.42      0.00            1.00   
25%        259000.00           161.78      0.00           21.00   
50%        385000.00           204.90      0.00           90.00   
75%        600000.00           303.46     64.00          178.00   
max      31500000.00         17326.73   2476.00         2727.00   

       SQUARE_FOOTAGE  BEDROOMS  BATHROOMS  
count        13428.00  13428.00   13428.00  
mean          1939.70      2.83       2.19  
std           6081.08      1.73       1.56  
min              0.00      0.00       0.00  
25%           1064.00      2.00       1.00  
50%           1749.00      3.00       2.00  
75%           247

In [7]:
print("=== Agent Performance (Avg DOM & Listing Volume) ===\n")

#  Join listings with agents on AGENT_ID
# Keep only useful columns + agent name
merged = listings.merge(
    agents[['AGENT_ID', 'AGENT_NAME']],
    on='AGENT_ID',
    how='left'   # keep all listings even if agent missing (rare)
)

# Group by agent and calculate stats
agent_stats = merged.groupby('AGENT_NAME').agg(
    num_listings=('LISTING_ID', 'count'),
    avg_dom=('DAYS_ON_MARKET', 'mean'),
    median_dom=('DAYS_ON_MARKET', 'median'),
    total_price=('CURRENT_PRICE', 'sum')
).reset_index()

# Filter to agents with at least 5 listings (avoids noise)
agent_stats = agent_stats[agent_stats['num_listings'] >= 5]

# Sort and show top/bottom
print("Top 5 agents by LOWEST average DOM (fastest sellers):")
print(agent_stats.sort_values('avg_dom').head(5)[['AGENT_NAME', 'num_listings', 'avg_dom', 'median_dom']].round(1))

print("\nBottom 5 agents by HIGHEST average DOM (slowest sellers):")
print(agent_stats.sort_values('avg_dom', ascending=False).head(5)[['AGENT_NAME', 'num_listings', 'avg_dom', 'median_dom']].round(1))

print(f"\nTotal agents with >=5 listings: {len(agent_stats)}")

=== Agent Performance (Avg DOM & Listing Volume) ===

Top 5 agents by LOWEST average DOM (fastest sellers):
            AGENT_NAME  num_listings  avg_dom  median_dom
336   Ana De Los Reyes             5      6.0         6.0
2912        Jason Long             5      7.0         8.0
5288      Nola J. Lusk             5      9.6         5.0
2907        Jason Holt             6     10.3        10.0
6615       Thao Nguyen             6     13.3        13.0

Bottom 5 agents by HIGHEST average DOM (slowest sellers):
               AGENT_NAME  num_listings  avg_dom  median_dom
3601     Karan Wethington            10   1471.2      1472.0
3804        Kelly Methvin            11    970.9      1243.0
7009         Wendi Conley             7    877.0       877.0
4294  Lisa Bransom-wright            14    703.3       637.0
3142       Jessica Knight            16    658.8       659.0

Total agents with >=5 listings: 309


In [8]:
print("=== County-Level Summary ===\n")

county_stats = listings.groupby('COUNTY').agg(
    num_listings=('LISTING_ID', 'count'),
    avg_price=('CURRENT_PRICE', 'mean'),
    median_price=('CURRENT_PRICE', 'median'),
    avg_dom=('DAYS_ON_MARKET', 'mean'),
    median_dom=('DAYS_ON_MARKET', 'median')
).reset_index()

# Sort by number of listings descending
county_stats = county_stats.sort_values('num_listings', ascending=False)

print("Top 10 counties by listing volume:")
print(county_stats.head(10)[['COUNTY', 'num_listings', 'avg_price', 'median_price', 'avg_dom', 'median_dom']].round(0))

print("\nBottom 5 counties (fewest listings, but still some activity):")
print(county_stats.tail(5)[['COUNTY', 'num_listings', 'avg_price', 'median_price', 'avg_dom', 'median_dom']].round(0))

=== County-Level Summary ===

Top 10 counties by listing volume:
       COUNTY  num_listings  avg_price  median_price  avg_dom  median_dom
28     Travis          3930   896095.0      524900.0    116.0        80.0
1       Bexar           939   389300.0      299990.0    105.0       163.0
12  Fort Bend           884   530801.0      413972.0     21.0        20.0
13  Galveston           720   492595.0      358000.0     89.0        88.0
10    El Paso           567   355391.0      290000.0     92.0        92.0
16     Harris           565   418839.0      350000.0     51.0         6.0
25     Parker           560   542972.0      282500.0    333.0       260.0
0        Bell           557   373116.0      299500.0     17.0        17.0
11      Ellis           549   770937.0      439000.0    239.0       173.0
18    Hidalgo           428   320336.0      239900.0     88.0        88.0

Bottom 5 counties (fewest listings, but still some activity):
        COUNTY  num_listings  avg_price  median_price  avg